In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
note_names = ['A_m', 'Ash_m', 'B_m', 'C', 'Csh', 'D', 'Dsh', 'E', 'F', 'Fsh', 'G', 'Gsh', 'A', 'Ash', 'B', 'C_p', 'Csh_p', 'D_p', 'Dsh_p', 'E_p']

note_form_style = ['sin', 'mndr', 'trin']


note_frequency_Hz = [220, 233.08, 246.94, 261.63, 277.18, 293.67, 311.13, 329.63, 349.23, 369.99, 392, 418.3, 440, 466.16, 493.88, 523.25, 554.36, 587.33, 622.26, 659.26]

DOR_K = 1241.212121

time_step = 1/48000

exp_step = 0.007


In [2]:
time_step

2.0833333333333333e-05

In [3]:
def calc_exp_form (teme_step = 0.007):
    note_form = []
    for t in range(0, 500):
        note_form.append(round((1 / np.exp(teme_step * t)), 5))
    return note_form

In [6]:
exp_fr = calc_exp_form()

print('const float EXPONENT [500] = {', end = '')

for i, exp_s in enumerate(exp_fr):
    temp_str = str(exp_s)
    
    spases = ',' + ' ' * (8 - len(temp_str))
    
    if ((i % 14) == 13):
        spases = ',\n' + ' ' * 30
    
    if (i == len(exp_fr)-1):
        print(temp_str, end='};')
    else:
        print(temp_str, end=spases)

const float EXPONENT [500] = {1.0,     0.99302, 0.9861,  0.97922, 0.97239, 0.96561, 0.95887, 0.95218, 0.94554, 0.93894, 0.93239, 0.92589, 0.91943, 0.91302,
                              0.90665, 0.90032, 0.89404, 0.88781, 0.88161, 0.87547, 0.86936, 0.86329, 0.85727, 0.85129, 0.84535, 0.83946, 0.8336,  0.82779,
                              0.82201, 0.81628, 0.81058, 0.80493, 0.79932, 0.79374, 0.7882,  0.7827,  0.77724, 0.77182, 0.76644, 0.76109, 0.75578, 0.75051,
                              0.74528, 0.74008, 0.73492, 0.72979, 0.7247,  0.71964, 0.71462, 0.70964, 0.70469, 0.69977, 0.69489, 0.69004, 0.68523, 0.68045,
                              0.6757,  0.67099, 0.66631, 0.66166, 0.65705, 0.65246, 0.64791, 0.64339, 0.6389,  0.63445, 0.63002, 0.62563, 0.62126, 0.61693,
                              0.61263, 0.60835, 0.60411, 0.5999,  0.59571, 0.59156, 0.58743, 0.58333, 0.57926, 0.57522, 0.57121, 0.56722, 0.56327, 0.55934,
                              0.55544, 0.55156, 0.54772, 0.54389

In [7]:
def calc_note_form_sin (note_mug, note_freq, DOR_K = 1241.212121, teme_step = 0.000025):
    note_form = []
    new_val = 0
    old_val = 0
    for t in np.arange(0, 0.01, time_step):
      new_val = round(note_mug * np.sin(np.pi * 2 * note_freq * t) * DOR_K)
      if (new_val >= 0 and old_val < 0):
        return note_form
      else:
        note_form.append(new_val)
        old_val = new_val
    
    

In [8]:
def calc_note_form_mndr (note_mug, note_freq, DOR_K = 1241.212121, teme_step = 0.000025):
    note_form = []
    new_val = 0
    old_val = 0
    for t in np.arange(0, 0.01, time_step):
      new_val = round(note_mug * (1 if np.sin(np.pi * 2 * note_freq * t) >= 0 else -1) * DOR_K)
      if (new_val >= 0 and old_val < 0):
        return note_form
      else:
        note_form.append(new_val)
        old_val = new_val

In [9]:


def calc_note_form_trn(note_mug, note_freq, bs, DOR_K = 1241.212121, teme_step = 0.000025):
    
    trin_form = np.arange(-1, 1, 2/bs)

    trin_form = trin_form * note_mug * DOR_K
    
    trin_form = trin_form[0: bs]
    
    return (np.around(trin_form)).astype(int)


In [10]:
def print_form_note(note_name, note_form, note_form_style, size_tabs = 40, end = '\n\n'):
    
    data_type = "const int32_t "
    define_form_string = data_type + note_name + '_form_' + note_form_style + ' [' + str(len(note_form)) + '] ='
    temp = len(define_form_string)
    define_form_string += ' ' * (size_tabs - temp - 1) + '{'
    
    print(define_form_string, end = '')
    
    for i, n in enumerate(note_form):
      if (n >= 0):
        if (n < 10):
          spases = ',    '
        elif (n < 100):
          spases = ',   '
        else:
          spases = ',  '
      else:
        if (n > -10):
          spases = ',   '
        elif (n > -100):
          spases = ',  '
        else:
          spases = ', '

      if ((i % 20) == 19):
        spases = ',\n' + ' ' * size_tabs

      if (i == len(note_form)-1):
        print(n, end='};')
      else:
        print(n, end=spases)
    
    print(end, end = '')


In [11]:
note_magnitude = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
sizes = []

for n, m, f in zip(note_names, note_magnitude, note_frequency_Hz):
    nt_fm = calc_note_form_sin(m, f)
    sizes.append(len(nt_fm))
    print_form_note(n, nt_fm, note_form_style[0])
    
    
print('const int32_t PAUSE_form_sin [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int32_t *note_sin_table[21] =    {' + n + '_form_' + note_form_style[0] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[0] + ',')
        
print(' ' * 40 + 'PAUSE_form_sin};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint8_t note_form_size[21] = ' + temp_str + ';')

const int32_t A_m_form_sin [219] =     {0,    7,    14,   21,   29,   36,   43,   50,   57,   64,   71,   77,   84,   91,   97,   104,  110,  117,  123,  129,
                                        135,  141,  147,  153,  158,  164,  169,  174,  179,  184,  189,  193,  198,  202,  206,  210,  214,  217,  221,  224,
                                        227,  230,  232,  235,  237,  239,  241,  242,  244,  245,  246,  247,  248,  248,  248,  248,  248,  248,  247,  246,
                                        245,  244,  243,  241,  239,  237,  235,  232,  230,  227,  224,  221,  218,  214,  210,  206,  202,  198,  194,  189,
                                        184,  180,  175,  169,  164,  159,  153,  147,  142,  136,  130,  124,  117,  111,  105,  98,   91,   85,   78,   71,
                                        64,   57,   50,   43,   36,   29,   22,   15,   8,    1,    -6,   -14,  -21,  -28,  -35,  -42,  -49,  -56,  -63,  -70,
                                        -77,  -

In [12]:
note_magnitude = [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]
sizes = []

for n, m, f in zip(note_names, note_magnitude, note_frequency_Hz):
    nt_fm = calc_note_form_mndr(m, f)
    sizes.append(len(nt_fm))
    print_form_note(n, nt_fm, note_form_style[1])
    
    
print('const int32_t PAUSE_form_mndr [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int32_t *note_mndr_table[21] =   {' + n + '_form_' + note_form_style[1] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[1] + ',')
        
print(' ' * 40 + 'PAUSE_form_mndr};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint8_t note_form_size[21] = ' + temp_str + ';')

const int32_t A_m_form_mndr [219] =    {186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,
                                        186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,
                                        186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,
                                        186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,
                                        186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  186,
                                        186,  186,  186,  186,  186,  186,  186,  186,  186,  186,  -186, -186, -186, -186, -186, -186, -186, -186, -186, -186,
                                        -186,

In [13]:
note_magnitude = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
buf_sizes = [219, 206, 195, 184, 174, 164, 155, 146, 138, 130, 123, 115, 110, 103, 98, 92, 87, 82, 78, 73, 30]
sizes = []

for n, m, f, bs in zip(note_names, note_magnitude, note_frequency_Hz, buf_sizes):
    nt_fm = calc_note_form_trn(m, f, bs)
    sizes.append(len(nt_fm))
    print_form_note(n, nt_fm, note_form_style[2])
    
    
print('const int32_t PAUSE_form_trin [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int32_t *note_trin_table[21] =   {' + n + '_form_' + note_form_style[2] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[2] + ',')
        
print(' ' * 40 + 'PAUSE_form_trin};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint8_t note_form_size[21] = ' + temp_str + ';')

const int32_t A_m_form_trin [219] =    {-248, -246, -244, -241, -239, -237, -235, -232, -230, -228, -226, -223, -221, -219, -217, -214, -212, -210, -207, -205,
                                        -203, -201, -198, -196, -194, -192, -189, -187, -185, -182, -180, -178, -176, -173, -171, -169, -167, -164, -162, -160,
                                        -158, -155, -153, -151, -148, -146, -144, -142, -139, -137, -135, -133, -130, -128, -126, -124, -121, -119, -117, -114,
                                        -112, -110, -108, -105, -103, -101, -99,  -96,  -94,  -92,  -90,  -87,  -85,  -83,  -80,  -78,  -76,  -74,  -71,  -69,
                                        -67,  -65,  -62,  -60,  -58,  -56,  -53,  -51,  -49,  -46,  -44,  -42,  -40,  -37,  -35,  -33,  -31,  -28,  -26,  -24,
                                        -22,  -19,  -17,  -15,  -12,  -10,  -8,   -6,   -3,   -1,   1,    3,    6,    8,    10,   12,   15,   17,   19,   22,
                                        24, 